In [2]:
import numpy as np 
import pandas as pd

In [3]:
df = pd.read_csv("./songrec/songdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.shape

(57650, 4)

In [5]:
df=df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [6]:
df['artist']

0                     Hollies
1                    Hillsong
2                     Roxette
3       Indiana Bible College
4                    Iggy Pop
                ...          
4995            Conway Twitty
4996              Johnny Cash
4997           Counting Crows
4998               Elton John
4999                Metallica
Name: artist, Length: 5000, dtype: object

In [7]:
df['song'][0]

'Boys In The Band'

In [8]:
df['text'][0]

"With Mr Soul or King Creole  \nYou gotta go with the man in the band  \nHe'll be your guide  \nSinging la la la la  \nBut what makes Mr Politic  \nA lot of words another trick  \nTurn peace into war  \nSinging ha ha ha ha  \n  \nOoh it's not the way we want it  \nNot the way at all  \nNot the way to change our minds  \nOur backs up against the wall  \nThe boys in the band all live in harmony  \nFor life's a poem  \nAnd we can make it rhyme  \nThe boys in the band all live in harmony  \nLet's get it together before we  \nGo down, down, down  \nI don't want to theorise it  \nJust hope people realise  \nThe simple melody can free your soul  \nAnd let you be man what you are  \nAnd fly la la la  \nBut every back street slum where  \nPushers push and hookers hook  \nJust won't set you free with ha ha ha ha  \nOooh it's not the way we want it  \nNot the way at all  \nNot the way to change our minds  \nOur backs up against the wall  \nThe boys in the band all live in harmony  \nFor life's a 

In [9]:
df['text']=df['text'].str.lower().replace(r'[^\w\s]', '').replace(r'\n',' ',regex=True)

In [10]:
df['text'][0]

"with mr soul or king creole   you gotta go with the man in the band   he'll be your guide   singing la la la la   but what makes mr politic   a lot of words another trick   turn peace into war   singing ha ha ha ha      ooh it's not the way we want it   not the way at all   not the way to change our minds   our backs up against the wall   the boys in the band all live in harmony   for life's a poem   and we can make it rhyme   the boys in the band all live in harmony   let's get it together before we   go down, down, down   i don't want to theorise it   just hope people realise   the simple melody can free your soul   and let you be man what you are   and fly la la la   but every back street slum where   pushers push and hookers hook   just won't set you free with ha ha ha ha   oooh it's not the way we want it   not the way at all   not the way to change our minds   our backs up against the wall   the boys in the band all live in harmony   for life's a poem   and we can make it rhyme 

In [11]:
df['text']

0       with mr soul or king creole   you gotta go wit...
1       anchor   i have this hope   as an anchor for m...
2       she knows i'd do anything she wants me to,   t...
3       chorus   (sopranos)   he reigns   let all eart...
4       watching the news   the plot you already know ...
                              ...                        
4995    before you take another step   there's somethi...
4996    six foot six he stood on the ground,   he weig...
4997    original by the ramones      twenty-twenty-twe...
4998    t seems as though i've lived my life   on the ...
4999    down in the depths of my fiery home   the summ...
Name: text, Length: 5000, dtype: object

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def tokenisation(txt):
    doc = nlp(txt)
    stemming = [token.lemma_ for token in doc]
    return " ".join(stemming)



In [13]:
df['text'] = df['text'].apply(lambda x: tokenisation(x))

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
tfidvector = TfidfVectorizer(analyzer='word', stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [16]:
similarity[0]

array([1.        , 0.02465926, 0.01997787, ..., 0.00153778, 0.07130997,
       0.03264626], shape=(5000,))

In [22]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [26]:
recommendation('Levitate')

['Love Or Nothing',
 'Those Were The Days',
 'Just Be Good To Me',
 'Bigger',
 'Ann',
 'Bridegroom',
 "Can't Get You Out Of My Head (Soulwax Elektronic M",
 'Soften Your Heart',
 "Look Who's Talking",
 'The Passion',
 'Glad Tidings',
 "I Can't Lie",
 "Love Is Teasin'",
 'Spell Of Desire',
 'Ooh La La',
 'How Can You Mend A Broken Heart',
 'Pretty Girl Milking A Cow',
 'To Live And Die In La',
 'Piano Man',
 'Fiesta De La Noche']

In [27]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl', 'wb'))